In [1]:
import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(kit_dir, '..'))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from dotenv import load_dotenv
from pprint import pprint

load_dotenv(os.path.join(repo_dir, '.env'))

True

In [2]:
from langchain.prompts import load_prompt
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import CommaSeparatedListOutputParser, StructuredOutputParser, ResponseSchema
from langchain_community.document_loaders import TextLoader
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from utils.vectordb.vector_db import VectorDb
from utils.model_wrappers.api_gateway import APIGateway
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain

# Model

In [3]:
# Model definition

model = APIGateway.load_llm(type='sncloud', select_expert='llama3-8b')

# Analysis Methods

In [4]:
def get_chunks(documents):
    # split long document
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
    return splitter.split_documents(documents)

## Reduce call method

In [5]:
def reduce_call(conversation):
    reduce_prompt = load_prompt(os.path.join(kit_dir, 'prompts/reduce.yaml'))
    reduce_chain = LLMChain(llm=model, prompt=reduce_prompt)
    combine_documents_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name='transcription_chunks')
    # Combines and iteravelly reduces the documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `StuffDocumentsChain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=1200,
    )
    print('reducing call')
    new_document = reduce_documents_chain.invoke(conversation)['output_text']
    print('call reduced')
    return new_document

## sumarization method

In [6]:
def get_summary(conversation, model=model):
    summarization_prompt = load_prompt(os.path.join(kit_dir, 'prompts/summarization.yaml'))
    output_parser = StrOutputParser()
    summarization_chain = summarization_prompt | model | output_parser
    input_variables = {'conversation': conversation}
    print('summarizing')
    summarization_response = summarization_chain.invoke(input_variables)
    print('summarizing done')
    return summarization_response

## main topic classification method

In [7]:
def classify_main_topic(conversation, classes, model=model):
    topic_classification_prompt = load_prompt(os.path.join(kit_dir, 'prompts/topic_classification.yaml'))
    list_output_parser = CommaSeparatedListOutputParser()
    list_format_instructions = list_output_parser.get_format_instructions()
    topic_classification_chain = topic_classification_prompt | model | list_output_parser
    input_variables = {
        'conversation': conversation,
        'topic_classes': '\n\t- '.join(classes),
        'format_instructions': list_format_instructions,
    }
    print('classification')
    topic_classification_response = topic_classification_chain.invoke(input_variables)
    print('classification done')
    return topic_classification_response

## named entity recognition method

In [8]:
def get_entities(conversation, entities, model=model):
    ner_prompt = load_prompt(os.path.join(kit_dir, 'prompts/ner.yaml'))
    response_schemas = []
    for entity in entities:
        response_schemas.append(ResponseSchema(name=entity, description=f'{entity}s find in conversation', type='list'))
    entities_output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    ner_chain = ner_prompt | model | entities_output_parser
    input_variables = {
        'conversation': conversation,
        'entities': '\n\t- '.join(entities),
        'format_instructions': entities_output_parser.get_format_instructions(),
    }
    print('extracting entities')
    ner_response = ner_chain.invoke(input_variables)
    print('extracting entities done')
    return ner_response

## sentiment analysis method

In [9]:
def get_sentiment(conversation, sentiments, model=model):
    sentiment_analysis_prompt = load_prompt(os.path.join(kit_dir, 'prompts/sentiment_analysis.yaml'))
    list_output_parser = CommaSeparatedListOutputParser()
    list_format_instructions = list_output_parser.get_format_instructions()
    sentiment_analysis_chain = sentiment_analysis_prompt | model | list_output_parser
    input_variables = {
        'conversation': conversation,
        'sentiments': sentiments,
        'format_instructions': list_format_instructions,
    }
    print('sentiment analysis')
    sentiment_analysis_response = sentiment_analysis_chain.invoke(input_variables)
    print('sentiment analysis done')
    return sentiment_analysis_response[0]

## factual check method

In [10]:
def set_retriever(documents_path, urls):
    print('setting retriever')
    vdb = VectorDb()
    retriever = vdb.create_vdb(
        documents_path, 1000, 200, 'faiss', None, load_txt=True, load_pdf=True, urls=urls
    ).as_retriever()
    print('retriever set')
    return retriever


def factual_accuracy_analysis(conversation, retriever, model=model):
    factual_accuracy_analysis_response_schemas = [
        ResponseSchema(name='correct', description='wether or not the provided information is correct', type='bool'),
        ResponseSchema(
            name='errors',
            description='list of summarized errors made by the agent, if there is no errors, empty list',
            type='list',
        ),
        ResponseSchema(
            name='score', description='punctuation from 1 to 100 of the overall quality of the agent', type='int'
        ),
    ]
    factual_accuracy_analysis_output_parser = StructuredOutputParser.from_response_schemas(
        factual_accuracy_analysis_response_schemas
    )
    format_instructions = factual_accuracy_analysis_output_parser.get_format_instructions()
    retrieval_qa_chat_prompt = load_prompt(os.path.join(kit_dir, 'prompts/factual_accuracy_analysis.yaml'))
    combine_docs_chain = create_stuff_documents_chain(model, retrieval_qa_chat_prompt)
    retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
    input_variables = {'input': conversation, 'format_instructions': format_instructions}
    print('factual check')
    model_response = retrieval_chain.invoke(input_variables)['answer']
    factual_accuracy_analysis_response = factual_accuracy_analysis_output_parser.invoke(model_response)
    print('factual check done')
    return factual_accuracy_analysis_response

## Procedural Analysis method
 

In [11]:
def procedural_accuracy_analysis(conversation, procedures_path, model=model):
    """
    Analyse the procedural accuracy of the given conversation.

    Args:
        conversation (str): The conversation to analyse.
        procedures_path (str): The path to the file containing the procedures.
        model (Langchain LLM Model, optional): The language model to use for summarization and classification.
            Defaults to a SambaNovaEndpoint model.
    Returns:
        dict: A dictionary containing the procedural accuracy analysis results. The keys are:
            - "correct": A boolean indicating whether the agent followed all the procedures.
            - "errors": A list of summarized errors made by the agent, if any.
            - "evaluation": A list of booleans evaluating if the agent followed each one of the procedures listed.
    """
    procedures_analysis_response_schemas = [
        ResponseSchema(name='correct', description='wether or not the agent followed all the procedures', type='bool'),
        ResponseSchema(
            name='errors',
            description='list of summarized errors made by the agent, if there is no errors, empty list',
            type='list',
        ),
        ResponseSchema(
            name='evaluation',
            description='list of booleans evaluating if the agent followed each one of the procedures listed',
            type='list[bool]',
        ),
    ]
    procedures_analysis_output_parser = StructuredOutputParser.from_response_schemas(
        procedures_analysis_response_schemas
    )
    format_instructions = procedures_analysis_output_parser.get_format_instructions()
    procedures_prompt = load_prompt(os.path.join(kit_dir, 'prompts/procedures_analysis.yaml'))
    with open(procedures_path, 'r') as file:
        procedures = file.readlines()
    procedures_chain = procedures_prompt | model | procedures_analysis_output_parser
    input_variables = {'input': conversation, 'procedures': procedures, 'format_instructions': format_instructions}
    print('procedures check')
    procedures_analysis_response = procedures_chain.invoke(input_variables)
    print('procedures check done')
    return procedures_analysis_response

## NPS prediction method

In [12]:
def get_nps(conversation, model=model):
    nps_response_schemas = [
        ResponseSchema(name='description', description='reasoning', type='str'),
        ResponseSchema(name='score', description='punctuation from 1 to 10 of the NPS', type='int'),
    ]
    nps_output_parser = StructuredOutputParser.from_response_schemas(nps_response_schemas)
    format_instructions = nps_output_parser.get_format_instructions()
    nps_prompt = load_prompt(os.path.join(kit_dir, 'prompts/nps.yaml'))
    nps_chain = nps_prompt | model | nps_output_parser
    input_variables = {'conversation': conversation, 'format_instructions': format_instructions}
    print(f'predicting nps')
    nps = nps_chain.invoke(input_variables)
    print(f'nps chain finished')
    return nps

## Quallity assement method

In [13]:
def get_call_quality_assessment(conversation, factual_result, procedures_result):
    total_score = 0
    # predict a NPS of the call
    nps = get_nps(conversation)
    total_score += nps['score'] * 10
    # include the factual analysis score
    total_score += factual_result['score']
    # include the procedures analysis score
    if len(procedures_result['evaluation']) == 0:
        total_score += 1
    else:
        total_score += procedures_result['evaluation'].count(True) / len(procedures_result['evaluation'])
    # Simple average
    overall_score = total_score / 3
    return overall_score

# complete analysis 

In [14]:
path = os.path.join(kit_dir, 'data/conversations/transcription')
conversations = os.listdir(path)
documents = []
for conversation in conversations:
    conversation_path = os.path.join(path, conversation)
    loader = TextLoader(conversation_path)
    documents.extend(loader.load())
documents

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
chunks = splitter.split_documents(documents)

In [ ]:
import concurrent.futures


def call_analysis_parallel(
    conversation, documents_path, facts_urls, procedures_path, classes_list, entities_list, sentiment_list
):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submitting tasks to executor
        reduced_conversation_future = executor.submit(reduce_call, conversation=conversation)
        retriever = set_retriever(documents_path=documents_path, urls=facts_urls)
        reduced_conversation = reduced_conversation_future.result()
        summary_future = executor.submit(get_summary, conversation=reduced_conversation)
        classification_future = executor.submit(
            classify_main_topic, conversation=reduced_conversation, classes=classes_list
        )
        entities_future = executor.submit(get_entities, conversation=reduced_conversation, entities=entities_list)
        sentiment_future = executor.submit(get_sentiment, conversation=reduced_conversation, sentiments=sentiment_list)
        factual_analysis_future = executor.submit(
            factual_accuracy_analysis, conversation=reduced_conversation, retriever=retriever
        )
        procedural_analysis_future = executor.submit(
            procedural_accuracy_analysis, conversation=reduced_conversation, procedures_path=procedures_path
        )

        # Retrieving results
        summary = summary_future.result()
        classification = classification_future.result()
        entities = entities_future.result()
        sentiment = sentiment_future.result()
        factual_analysis = factual_analysis_future.result()
        procedural_analysis = procedural_analysis_future.result()
    quality_score = get_call_quality_assessment(reduced_conversation, factual_analysis, procedural_analysis)

    return {
        'summary': summary,
        'classification': classification,
        'entities': entities,
        'sentiment': sentiment,
        'factual_analysis': factual_analysis,
        'procedural_analysis': procedural_analysis,
        'quality_score': quality_score,
    }


classes = ['medical emergency', 'animals emergency', 'terrorism emergency', 'fire emergency', 'undefined']
entities = ['city', 'address', 'customer_name', 'payment_type']
sentiments = ['positive', 'negative', 'neutral']
pprint(
    call_analysis_parallel(
        conversation=chunks,
        documents_path=os.path.join(kit_dir, 'data/documents'),
        facts_urls=[],
        procedures_path=os.path.join(kit_dir, 'data/documents/example_procedures.txt'),
        classes_list=classes,
        entities_list=entities,
        sentiment_list=sentiments,
    )
)